In [81]:
import pandas as pd
import sqlite3

# NYC Dog Licensing Dataset (ETL): Extract

In [82]:
#pull and save NYC Dog Licensing Dataset from NYC Open Data
dogs = pd.read_csv("../Data/NYC_Dog_Licensing_Dataset.csv")
print(dogs.shape)
dogs.head()

(121949, 15)


,RowNumber,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,CommunityDistrict,CensusTract2010,NTA,CityCouncilDistrict,CongressionalDistrict,StateSenatorialDistrict,LicenseIssuedDate,LicenseExpiredDate
0,533,BONITA,F,05/01/2013 12:00:00 AM,Unknown,Queens,11435.0,412.0,208.0,QN61,28.0,5.0,10.0,10/24/2014,11/15/2017
1,548,ROCKY,M,05/01/2014 12:00:00 AM,Labrador Retriever Crossbreed,Queens,11691.0,414.0,100801.0,QN15,31.0,5.0,10.0,10/25/2014,10/25/2019
2,622,BULLY,M,07/01/2010 12:00:00 AM,American Pit Bull Terrier/Pit Bull,Queens,11419.0,410.0,98.0,QN55,28.0,5.0,10.0,10/28/2014,09/24/2016
3,633,COCO,M,02/01/2005 12:00:00 AM,Labrador Retriever,Queens,11692.0,414.0,964.0,QN12,31.0,5.0,10.0,10/29/2014,10/29/2017
4,655,SKI,F,09/01/2012 12:00:00 AM,American Pit Bull Terrier/Pit Bull,Queens,11691.0,414.0,100802.0,QN15,31.0,5.0,10.0,10/31/2014,10/31/2019


# NYC Dog Licensing Dataset (ETL): Transform

In [83]:
#drop extraneous columns of dataset
dogs = dogs.drop(['RowNumber',
                  'AnimalBirthMonth',
                  'CommunityDistrict',
                  'CensusTract2010',
                  'NTA',
                  'CityCouncilDistrict',
                  'CongressionalDistrict',
                  'StateSenatorialDistrict'], axis=1)

#drop records with missing data
dogs = dogs.dropna()
print(dogs.shape)

#extract license issued and expired years
dogs['LicenseIssuedYear'] = pd.DatetimeIndex(dogs['LicenseIssuedDate']).year
dogs['LicenseExpiredYear'] = pd.DatetimeIndex(dogs['LicenseExpiredDate']).year

#drop license issued and expired date columns of dataset
dogs = dogs.drop(['LicenseIssuedDate', 'LicenseExpiredDate'], axis=1)

#set integer types
dogs.ZipCode = dogs.ZipCode.astype(int)
dogs.LicenseIssuedYear = dogs.LicenseIssuedYear.astype(int)
dogs.LicenseExpiredYear = dogs.LicenseExpiredYear.astype(int)

(121713, 7)


In [84]:
#review cleaned dataset
dogs.head()

,AnimalName,AnimalGender,BreedName,Borough,ZipCode,LicenseIssuedYear,LicenseExpiredYear
0,BONITA,F,Unknown,Queens,11435,2014,2017
1,ROCKY,M,Labrador Retriever Crossbreed,Queens,11691,2014,2019
2,BULLY,M,American Pit Bull Terrier/Pit Bull,Queens,11419,2014,2016
3,COCO,M,Labrador Retriever,Queens,11692,2014,2017
4,SKI,F,American Pit Bull Terrier/Pit Bull,Queens,11691,2014,2019


In [85]:
#check data types of dataset
dogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121713 entries, 0 to 121948
Data columns (total 7 columns):
AnimalName            121713 non-null object
AnimalGender          121713 non-null object
BreedName             121713 non-null object
Borough               121713 non-null object
ZipCode               121713 non-null int64
LicenseIssuedYear     121713 non-null int64
LicenseExpiredYear    121713 non-null int64
dtypes: int64(3), object(4)
memory usage: 7.4+ MB


In [86]:
#statistical description of dataset
dogs.describe()

,ZipCode,LicenseIssuedYear,LicenseExpiredYear
count,121713.000000,121713.000000,121713.000000
mean,10677.812945,2015.612934,2016.898524
std,917.666855,0.518188,0.919029
min,121.000000,2014.000000,2016.000000
25%,10029.000000,2015.000000,2016.000000
50%,10465.000000,2016.000000,2017.000000
75%,11228.000000,2016.000000,2017.000000
max,94608.000000,2016.000000,2022.000000


# NYC Dog Licensing Dataset (ETL): Load

In [87]:
#creating SQL connection
conn = sqlite3.connect('../Data/pet_care_industry.db')
c = conn.cursor()

#function to create table
def create_table(query):
    c.execute(query)

#function to close connection
def close_c_conn():
    c.close()
    conn.close()

In [88]:
#create dogs table
create_query = """CREATE TABLE dogs
                (id INTEGER PRIMARY KEY,
                 AnimalName TEXT,
                 AnimalGender TEXT,
                 BreedName TEXT,
                 Borough TEXT,
                 ZipCode INTEGER,
                 LicenseIssuedYear INTEGER,
                 LicenseExpiredYear INTEGER);"""

c.execute('DROP TABLE IF EXISTS dogs')
create_table(create_query)

In [90]:
#function to insert dogs into table
def insert_dogs(dogs):
    for i in range(len(dogs.index)):
        c.execute("""INSERT INTO dogs
                  (id,
                   AnimalName,
                   AnimalGender,
                   BreedName,
                   Borough,
                   ZipCode,
                   LicenseIssuedYear,
                   LicenseExpiredYear)
                   VALUES
                   (?,?,?,?,?,?,?,?)""",
                   (i,
                    dogs.iloc[i]['AnimalName'],
                    dogs.iloc[i]['AnimalGender'],
                    dogs.iloc[i]['BreedName'],
                    dogs.iloc[i]['Borough'],
                    int(dogs.iloc[i]['ZipCode']),
                    int(dogs.iloc[i]['LicenseIssuedYear']),
                    int(dogs.iloc[i]['LicenseExpiredYear'])))
        
    conn.commit()
    
#insert dogs into table
insert_dogs(dogs)

In [91]:
#check SQL dogs table
pd.read_sql_query("SELECT * FROM dogs;", conn)

,id,AnimalName,AnimalGender,BreedName,Borough,ZipCode,LicenseIssuedYear,LicenseExpiredYear
0,0,BONITA,F,Unknown,Queens,11435,2014,2017
1,1,ROCKY,M,Labrador Retriever Crossbreed,Queens,11691,2014,2019
2,2,BULLY,M,American Pit Bull Terrier/Pit Bull,Queens,11419,2014,2016
3,3,COCO,M,Labrador Retriever,Queens,11692,2014,2017
4,4,SKI,F,American Pit Bull Terrier/Pit Bull,Queens,11691,2014,2019
5,5,CHASE,M,Shih Tzu,Queens,11692,2014,2016
6,6,CHEWY,M,Shih Tzu,Queens,11694,2014,2016
7,7,CHASE,M,Labrador Retriever,Queens,11691,2014,2017
8,8,MILEY,F,Boxer,Queens,11419,2014,2016
9,9,KENZI,F,"Schnauzer, Miniature",Queens,11420,2014,2016


In [92]:
#close connection
close_c_conn()